# Fantasy Baseball Pandas Analysis

This takes a number of inputs and produces fantasy scores for each player.

In [2]:
import requests
import csv
import datetime
import subprocess
import os
import pandas as pd
import numpy as np
import seaborn as sb
from bs4 import BeautifulSoup
import sqlalchemy
import psycopg2
pd.options.display.max_columns = 150

/Users/zach/Library/Python/3.7/lib/python/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
# connection information for the database

POSTGRES_USER = os.environ.get('POSTGRES_USER')
POSTGRES_PASSWORD = os.environ.get('POSTGRES_PASSWORD')
POSTGRES_IP = "192.168.0.118"
POSTGRES_PORT = 5432
POSTGRES_DB = 'postgres'

## Get Roster Data

This will rip the roster information from ESPN and save it to a local CSV file.

In [3]:
LEAGUE_URL = "http://games.espn.com/flb/leaguerosters?leagueId={league_id}"
LEAGUE_ID = 15594

# translate ESPN names to Fangraphs names.
# Add new names as required, since name is used to join the data sets.
TRANSLATIONS = {
    'Nicky Delmonico': 'Nick Delmonico',
    'Yuli Gurriel': 'Yulieski Gurriel'
}


rosters_html = requests.get(LEAGUE_URL.format(league_id=LEAGUE_ID)).text
rosters_soup = BeautifulSoup(rosters_html, "lxml")

rosters = rosters_soup.find_all("table", {'class': 'playerTableTable'})

players = []
for roster in rosters:
    team_name = roster.find("a").text
    players_html = roster.find_all("td", {'class': 'playertablePlayerName'})
    for player in players_html:
        # parse player info
        player_name = player.text.split(",")[0]
        player_name = player_name.replace("*", "")

        # translate player name if necessary
        translation = TRANSLATIONS.get(player_name)
        if translation:
            player_name = translation

        # add to output list
        players.append([player_name, team_name])

with open("rosters.csv", "w", newline='') as out_file:
    writer = csv.writer(out_file)
    writer.writerow(("Name", "Squad"))
    writer.writerows(players)

## Get and Parse Actuals

Looks through the URLs to grab batting & pitching actuals and deliver those back to the user.

In [4]:
# static urls
season = datetime.datetime.now().year
PITCHERS_URL = "https://www.fangraphs.com/leaders.aspx?pos=all&stats=pit&lg=all&qual=0&type=c,36,37,38,40,-1,120,121,217,-1,24,41,42,43,44,-1,117,118,119,-1,6,45,124,-1,62,122,13&season={season}&month=0&season1={season}&ind=0&team=0&rost=0&age=0&filter=&players=0&page=1_100000".format(season=season)
BATTERS_URL = "https://www.fangraphs.com/leaders.aspx?pos=all&stats=bat&lg=all&qual=0&type=8&season={season}&month=0&season1={season}&ind=0&team=0&rost=0&age=0&filter=&players=0&page=1_10000".format(season=season)

# # request the data
pitchers_html = requests.get(PITCHERS_URL).text
batters_html = requests.get(BATTERS_URL).text

Now take the requests and parse out the relevant header information for each of the positions. This function will take one of the fangraphs pages as input and write out a CSV of that information once it's parsed.

In [6]:
def parse_array_from_fangraphs_html(input_html, out_file_name):
    """
    Take a HTML stats page from fangraphs and parse it out to a CSV file.
    """
    # parse input
    soup = BeautifulSoup(input_html, "lxml")
    table = soup.find("table", {"class": "rgMasterTable"})
    
    # get headers
    headers_html = table.find("thead").find_all("th")
    headers = []
    for header in headers_html:
        headers.append(header.text)
    
    # get rows
    rows = []
    rows_html = table.find("tbody").find_all("tr")
    for row in rows_html:
        row_data = []
        for cell in row.find_all("td"):
            row_data.append(cell.text)
        rows.append(row_data)
    
    # write to CSV file
    with open(out_file_name, "w") as out_file:
        writer = csv.writer(out_file)
        writer.writerow(headers)
        writer.writerows(rows)

Now that we have all of the player data, I'm writing these out to a CSV file if I want to load them again later without having to run the requests to those pages once more.

In [8]:
parse_array_from_fangraphs_html(batters_html, 'batters_actuals.csv')
parse_array_from_fangraphs_html(pitchers_html, 'pitchers_actuals.csv')

## Get Projections

For this part, we need to call some external bash code here, because the form data is too big to reasonably bring into the script here. Check out the [original blog post](https://zmsy.co/blog/fantasy-baseball/) on how to configure this for your own purposes.

In [7]:
subprocess.call('./get_fangraphs.sh', shell=True)

0

## Read Data Into Pandas

Load those CSV files using read_csv() in pandas. Since some of the percentage values are stored as strings, we need to parse those into floats.

We want to create two dataframes here:

- `dfb` - Batters Dataframe
- `dfp` - Pitchers Dataframe

In [11]:
dfb_act = pd.read_csv('batters_actuals.csv')

In [10]:
df_rost = pd.read_csv('rosters.csv')
dfb_act = pd.read_csv('batters_actuals.csv')
dfp_act = pd.read_csv('pitchers_actuals.csv')

# create a function to parse out percentage strings to floats
def parse_pctg(value):
    return float(value.split()[0]) / 100

# apply that to all percentage values in the dataframes
for col in dfb_act.columns:
    if '%' in col:
        dfb_act[col] = dfb_act[col].apply(lambda x: parse_pctg(x))
        
for col in dfp_act.columns:
    if '%' in col:
        dfp_act[col] = dfp_act[col].apply(lambda x: parse_pctg(x))

# rename columns to remove % (causes issues with postgres insert)
df_rost.columns = [x.lower() for x in df_rost.columns]
dfb_act.columns = [x.replace("%", "_pct").replace('+', '_plus').replace("/", "-").lower() for x in dfb_act.columns]
dfp_act.columns = [x.replace("%", "_pct").replace('+', '_plus').replace("/", "-").lower() for x in dfp_act.columns]

# change siera to numeric, doesn't get read correctly
dfp_act['siera'] = dfp_act['siera'].apply(pd.to_numeric, errors='coerce')

AttributeError: 'float' object has no attribute 'split'

In [8]:
with open('batters_projections.html', 'r') as bhtml:
    btxt = bhtml.read()
    dfb_proj = pd.read_html(btxt)[-1]  # read_html returns ALL tables, we just want the last one.
    dfb_proj.dropna(axis=1, inplace=True)

with open('pitchers_projections.html', 'r') as phtml:
    ptxt = phtml.read()
    dfp_proj = pd.read_html(ptxt)[-1]
    dfp_proj.dropna(axis=1, inplace=True)

# rename columns and apply naming scheme
dfb_proj.columns = [x.replace("%", "_pct").replace('/', '-').lower() for x in dfb_proj.columns]
dfp_proj.columns = [x.replace("%", "_pct").replace('/', '-').lower() for x in dfp_proj.columns]

In [13]:
dfb_proj.sort_values(by="woba", ascending=False)
dfb_proj["rc"] = dfb_proj["ab"] * dfb_proj["obp"] * dfb_proj["slg"]

In [15]:
dfb_proj.sort_values(by="rc", ascending=False)

,name,g,pa,ab,h,2b,3b,hr,r,rbi,bb,so,hbp,sb,cs,avg,obp,slg,ops,woba,fld,bsr,war,adp,rc
0,Mike Trout,159,686,538,159,30,3,45,124,115,128,140,13,17,4,0.296,0.439,0.614,1.053,0.427,-0.9,3.4,9.1,999.0,145.015748
4,Christian Yelich,154,665,568,172,35,4,36,108,106,83,137,7,24,5,0.304,0.397,0.569,0.966,0.397,0.8,4.0,5.7,999.0,128.307224
9,Juan Soto,156,672,555,163,32,4,36,108,118,109,127,3,10,3,0.294,0.410,0.561,0.971,0.402,-1.5,0.6,5.4,999.0,127.655550
2,Cody Bellinger,152,658,560,162,32,4,42,107,121,90,125,4,14,4,0.290,0.389,0.585,0.973,0.394,3.4,1.5,6.5,999.0,127.436400
13,Nolan Arenado,157,679,601,179,37,3,40,103,121,68,108,4,3,2,0.297,0.369,0.570,0.938,0.383,7.5,0.2,5.1,999.0,126.408330
23,Freddie Freeman,156,672,578,171,38,3,33,101,109,82,128,7,7,3,0.296,0.387,0.543,0.931,0.383,3.4,0.3,4.4,999.0,121.461498
49,J.D. Martinez,152,658,580,172,35,2,38,99,115,69,153,4,3,1,0.297,0.372,0.562,0.934,0.382,0.0,-2.4,3.4,999.0,121.257120
1,Alex Bregman,152,658,548,158,38,2,33,107,105,95,86,9,8,3,0.287,0.398,0.546,0.943,0.393,-0.3,0.3,6.6,999.0,119.084784
8,Mookie Betts,154,665,575,165,37,4,31,114,88,77,95,5,18,4,0.287,0.373,0.530,0.903,0.374,6.5,4.2,5.4,999.0,113.671750
31,Anthony Rizzo,156,672,567,161,33,3,31,94,106,78,95,22,6,4,0.284,0.388,0.516,0.904,0.378,4.2,-2.2,4.1,999.0,113.517936


In [11]:
# join the datasets together. we want one
# for batters and one for pitchers, with
# roster information in both of them.

dfb = pd.merge(dfb_proj, df_rost, how='left', on='name', suffixes=('.p', '.r'))
dfb = pd.merge(dfb, dfb_act, how='left', on='name', suffixes=('', '.a'))

dfp = pd.merge(dfp_proj, df_rost, how='left', on='name', suffixes=('.p', '.r'))
dfp = pd.merge(dfp, dfp_act, how='left', on='name', suffixes=('', '.a'))

## Filter and Qualify Information

The dataframes for pitchers/batters contain a lot of noise for things that we don't really care about, or won't actually have much of an effect on our league.

In [12]:
# apply some filters so we can get rid of players who won't play.
# minimum plate appearances or innings pitched

dfb = dfb[dfb['pa'] > 100]
dfp = dfp[dfp['ip'] > 20]

# add in league information
LEAGUES = {
    'Angels': 'AL',
    'Astros': 'AL',
    'Athletics': 'AL',
    'Blue Jays': 'AL',
    'Braves': 'NL',
    'Brewers': 'NL',
    'Cardinals': 'NL',
    'Cubs': 'NL',
    'Diamondbacks': 'NL',
    'Dodgers': 'NL',
    'Giants': 'NL',
    'Indians': 'AL',
    'Mariners': 'AL',
    'Marlins': 'NL',
    'Mets': 'NL',
    'Nationals': 'NL',
    'Orioles': 'AL',
    'Padres': 'NL',
    'Phillies': 'NL',
    'Pirates': 'NL',
    'Rangers': 'AL',
    'Rays': 'AL',
    'Red Sox': 'AL',
    'Reds': 'NL',
    'Rockies': 'NL',
    'Royals': 'AL',
    'Tigers': 'AL',
    'Twins': 'AL',
    'White Sox': 'AL',
    'Yankees': 'AL'
}


# derive the league for each player. my league is AL-only, so we want to focus on that.
def league(x):
    return LEAGUES.get(x)

dfb['league'] = dfb['team'].apply(lambda x: league(x))
dfp['league'] = dfp['team'].apply(lambda x: league(x))

In [13]:
# rearrange columns for readability and filter some out
# keep only ones relevant for our league
dfb_columns = ['name', 'squad', 'team', 'league', 'pa', 'pa.a', 'ab', 'h', 'so', 'k_pct', 'hr', 'hr.a', 'avg',
     'iso', 'babip', 'wrc_plus', 'avg.a', 'obp', 'obp.a', 'woba', 'woba.a', 'slg', 'slg.a', 'ops', 'bb_pct', 'bb']

dfp_columns = ['name', 'squad', 'team', 'league', 'ip', 'era', 'er', 'hr', 'so', 'bb', 'whip', 'k-9', 'bb-9', 'fip',
     'k-9.a', 'bb-9.a', 'k-bb', 'k_pct', 'whip.a', 'so.a', 'lob_pct', 'era.a', 'fip.a', 'e-f', 'xfip', 'siera', 'ip.a']


dfb = dfb[dfb_columns]
dfp = dfp[dfp_columns]

## Calculate Scores

The individual players in both the batting and pitching groups will get scored based on the entirety of the sample available. We calculate a composite score by taking the individual z-scores in each of the categories and trying to determine which players are above average.

In [14]:
# a 1 represents a positive number, i.e. higher is better
# a -1 represents negative, meaning lower is better

dfb_score_cols = {
    'pa': 1,
    'k_pct': -1,
    'hr': 1,
    'iso': 1,
    'obp': 1,
    'woba': 1,
    'woba.a': 1,
    'slg': 1
}

dfp_score_cols = {
    'ip': 1,
    'era': -1,
    'hr': -1,
    'so': 1,
    'whip': -1,
    'fip': -1,
    'k-9': 1,
    'siera': -1
}

for col in dfb_score_cols.keys():
    col_score = col + "_score"
    dfb[col_score] = (dfb[col] - dfb[col].mean()) / dfb[col].std(ddof=0) * dfb_score_cols.get(col)
    

for col in dfp_score_cols.keys():
    col_score = col + "_score"
    dfp[col_score] = (dfp[col] - dfp[col].mean()) / dfp[col].std(ddof=0) * dfp_score_cols.get(col)

## Write Information Back to Database

Once the table is available in the database, then we can query it again using other tools to make our lives easier. Then it can be used to display the information about each player in the Superset DB.

In [15]:
engine = sqlalchemy.create_engine("postgres://{user}:{password}@{host}:{port}/{db}".format(
    user=POSTGRES_USER,
    password=POSTGRES_PASSWORD,
    host=POSTGRES_IP,
    port=POSTGRES_PORT,
    db=POSTGRES_DB
))
conn = engine.connect()

OperationalError: (psycopg2.OperationalError) FATAL:  password authentication failed for user "None"
FATAL:  password authentication failed for user "None"
 (Background on this error at: http://sqlalche.me/e/e3q8)

In [16]:
dfb.to_sql('batters', conn, schema='fantasy', if_exists='replace')
dfb.to_sql('pitchers', conn, schema='fantasy', if_exists='replace')

NameError: name 'conn' is not defined

In [87]:
conn.close()